<center><h1>QBUS6860 - Visual Data Analytics</h1></center>

# Tutorial 08 - Interactive visualizations and dashboards

## Learning objectives and tasks:

1. Produce a dynamic map to present historical land temperature
2. Expand on an existing script of a dashboard created using <code>dash</code>
3. Demonstration: Create a standalone interactive dashboard with <code>bokeh</code>

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

## Task 1: Dynamic map to present historical land temperature globally

### Step 1: Importing Data 


In [2]:
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


### Step 2: Let's explore the data a bit more

In [3]:
print(df.head())

           dt  AverageTemperature  AverageTemperatureUncertainty Country
0  1743-11-01               4.384                          2.294   Åland
1  1743-12-01                 NaN                            NaN   Åland
2  1744-01-01                 NaN                            NaN   Åland
3  1744-02-01                 NaN                            NaN   Åland
4  1744-03-01                 NaN                            NaN   Åland


In [4]:
print(df.tail())

                dt  AverageTemperature  AverageTemperatureUncertainty  \
577457  2013-05-01              19.059                          1.022   
577458  2013-06-01              17.613                          0.473   
577459  2013-07-01              17.000                          0.453   
577460  2013-08-01              19.759                          0.717   
577461  2013-09-01                 NaN                            NaN   

         Country  
577457  Zimbabwe  
577458  Zimbabwe  
577459  Zimbabwe  
577460  Zimbabwe  
577461  Zimbabwe  


In [5]:
df.isnull().sum()

dt                                   0
AverageTemperature               32651
AverageTemperatureUncertainty    31912
Country                              0
dtype: int64

### Step 3: Let's clean the data

#### 1) We will drop “AverageTemperatureUncertainty” as we will not need it

In [6]:
df = df.drop("AverageTemperatureUncertainty", axis=1)

#### 2) We will rename columns to have a better workable columns

In [7]:
df = df.rename(columns={'dt':'Date'})
df = df.rename(columns={'AverageTemperature':'AvTemp'})

#### 3) Finally, we will drop the rows with the null values  

We have around 32000 rows with null values in AverageTemperature column. And in total we have around 577000 rows, so dropping them will not affect our analysis

In [8]:
df = df.dropna()

#### 4) Now we cleansed the data so let's have a look at the new dataset to check it

I like to look at first few rows of my data (you might want to look at fewer rows)

In [9]:
df.head(10)

,Date,AvTemp,Country
0,1743-11-01,4.384,Åland
5,1744-04-01,1.530,Åland
6,1744-05-01,6.702,Åland
7,1744-06-01,11.609,Åland
8,1744-07-01,15.342,Åland
10,1744-09-01,11.702,Åland
11,1744-10-01,5.477,Åland
12,1744-11-01,3.407,Åland
13,1744-12-01,-2.181,Åland
14,1745-01-01,-3.850,Åland


### Step 4: Now we need to filter data

#### 1) We will group the dataframe by Country name and the date.  
#### 2) We will also sort the values by date from latest to earliest time (luckily in Python we can do it together with grouping operation 1) above.



In [10]:
df_countries = df.groupby(['Country', 'Date']).sum().reset_index().sort_values('Date', ascending=False)
df_countries.head(10)

,Country,Date,AvTemp
318859,Mexico,2013-09-01,24.990
396261,Puerto Rico,2013-09-01,28.048
89235,Canada,2013-09-01,7.922
250579,Jamaica,2013-09-01,28.398
117566,Cuba,2013-09-01,28.424
418707,Saint Pierre And Miquelon,2013-09-01,13.745
507893,Turks And Caicas Islands,2013-09-01,29.770
144806,El Salvador,2013-09-01,26.405
73944,British Virgin Islands,2013-09-01,28.991
39448,Bahamas,2013-09-01,28.657


Our historical climate data is showing temperatures for all months between 1744 to 2013.  That's too much data, so let's work with a selection from 2005 to 2013.
You can select longer period, but it will take time to get a map.  If getting a map takes too long, select a shorter time period (play around, try different periods)

In [11]:
start_date = '2005-01-01'
end_date = '2013-01-01'

mask = (df_countries['Date'] > start_date) & (df_countries['Date'] <= end_date)
df_countries = df_countries.loc[mask]
df_countries.head(5)

,Country,Date,AvTemp
199293,Grenada,2013-01-01,26.579
476632,Sweden,2013-01-01,-7.811
418699,Saint Pierre And Miquelon,2013-01-01,-4.640
3971,Africa,2013-01-01,22.138
31324,Australia,2013-01-01,29.861


### Step 5: Overall visualisation

In [12]:
# Reorder the dataframe by ascending order of date and country name
df_countrydate = df_countries.groupby(['Date','Country']).sum().reset_index()
df_countrydate.head(5)

,Date,Country,AvTemp
0,2005-02-01,Afghanistan,1.822
1,2005-02-01,Africa,23.878
2,2005-02-01,Albania,2.257
3,2005-02-01,Algeria,13.500
4,2005-02-01,American Samoa,28.000


**Exercise 1:** Using the image of the dynamic map below as a guide, complete the blanks in the codes:

<img src = 'tutorial7_task1.png' />

In [13]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Create the visualization
# For more information: https://plotly.com/python-api-reference/generated/plotly.express.choropleth.html
fig = px.choropleth(df_countrydate,
    locations="       ",
    locationmode = "             ",
    color="    ",
    hover_name="Country",
    animation_frame="    "
    )

fig.update_layout(
    title_text = '                ',
    title_x = 0.5,
    geo=dict(
        showframe = ,
        showcoastlines = False,
        ))

fig.show() 

SyntaxError: invalid syntax (<ipython-input-13-d30fb74af4f0>, line 20)

In [ ]:
# Solutions
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Create the visualization
fig = px.choropleth(df_countrydate,
    locations="Country",
    locationmode = "country names",
    color="AvTemp",
    hover_name="Country",
    animation_frame="Date"
    )

fig.update_layout(
    title_text = 'Average Monthly Temperature Change 2005-2013',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
        ))

fig.show() 

> Ask yourself:
> 1. What is your experience like when you watched the dynamic map play?
> 2. What would you change in this dynamic map?

## Task 2: Expand on an existing script of a dashboard created using <code>dash</code>

In this task, we will learn to expand the functionality and the look of the dashboard that was presented during Lecture 7. As a quick re-cap, the original underlying dataframe behind the dashboard, and the content of the dashboard (which changes according to the slider) are seen below:

In [ ]:
df = px.data.gapminder()
df.head(3)

<img src = 'tutorial7_lecture.png' />

**Note:** The script below, adapted from Lecture 7 is <u>**not**</u> meant to be executed in this Jupyter notebook. Refer to slide 23 of Lecture 7 for instructions.

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

# The layout describes what the app looks like using HTML. The layout is a hierarchical tree of components
# For more information: https://dash.plotly.com/dash-html-components
app.layout = html.Div([

    # This is an interactive graph component. 
    # For more information: https://dash.plotly.com/interactive-graphing & https://dash.plotly.com/dash-core-components/graph
    dcc.Graph(id="scatter-plot"),
    html.P("Select a year:"),
    dcc.Slider(1952,2007,5,
        value = 1952,
        id='year-slider',
        marks={i: '{}'.format(i) for i in range(1952,2007,5)}
    )
])

# This is a callback decorator, which functions to change an output based on an input 
# The component_id and component_property keywords are optional
# For more information: https://dash.plotly.com/basic-callbacks
@app.callback(
    Output(component_id="scatter-plot", component_property="figure"), 
    Input(component_id="year-slider", component_property="value")
    )

def update_plot(year, continent):
    
    # replace with your own data source
    gm = px.data.gapminder()
    
    # using filtered data frame
    gmy = gm[gm['year']==year]

    # standard plotly graph - plotly documentation includes template codes for Dash
    fig = px.scatter( 
        gmy, y='lifeExp', x='gdpPercap',
        size='pop', color= 'continent', hover_data = ['country'],
        log_x=True, log_y = True,
        title = 'Health vs. Wealth across the world in '+str(year)
    )
    return fig

app.run_server(debug=True)

**Exercise 2:** Fill in the blanks below (and run the script in your terminal separately) to create an updated dashboard which now contains:

1. A markdown for description
2. Checkboxes for each continent
3. Actions in checkboxes that update the underlying dataframe

<img src = 'tutorial7_task2.png' />

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

##### Fill in the blanks with your code here #####
markdown_text = '''


'''

# The layout describes what the app looks like using HTML. The layout is a hierarchical tree of components
# For more information: https://dash.plotly.com/dash-html-components
app.layout = html.Div([

    # This is a markdown component, a Dash Core Component. DCCs are interactive components
    # For more information: https://dash.plotly.com/dash-core-components
    dcc.Markdown(children=             ),                                       ##### Fill in the blanks with your code here #####

    # This is an interactive graph component. 
    # For more information: https://dash.plotly.com/interactive-graphing & https://dash.plotly.com/dash-core-components/graph
    dcc.Graph(id="scatter-plot"),
    html.P("Select a year:"),
    dcc.Slider(1952,2007,5,
        value = 1952,
        id='year-slider',
        marks={i: '{}'.format(i) for i in range(1952,2007,5)}
    ),
    
    html.Br(),                                                                       
    html.Label("                                                 :"),           ##### Fill in the blanks with your code here ##### 
    dcc.Checklist(options=[                                                 ],  ##### Fill in the blanks with your code here ##### 
        value=[                                                 ],              ##### Fill in the blanks with your code here ##### 
        inline=True,                                                                 
        id='continent-slider'),                                                      
])

# This is a callback decorator, which functions to change an output based on an input 
# The component_id and component_property keywords are optional
# For more information: https://dash.plotly.com/basic-callbacks
@app.callback(
    Output(component_id="scatter-plot", component_property="figure"), 
    Input(component_id="year-slider", component_property="value"),
    Input(component_id="                ", component_property="value")          ##### Fill in the blanks with your code here #####
    )

def update_plot(year, continent):
    
    # replace with your own data source
    gm = px.data.gapminder()
    
    # using filtered data frame
    gmy = gm[(gm['year']==year) & (gm['continent'].isin(         ))]            ##### Fill in the blanks with your code here #####

    # standard plotly graph - plotly documentation includes template codes for Dash
    fig = px.scatter( 
        gmy, y='lifeExp', x='gdpPercap',
        size='pop', color= 'continent', hover_data = ['country'],
        log_x=True, log_y = True,
        title = 'Health vs. Wealth across the world in '+str(year)
    )
    return fig

app.run_server(debug=True)

In [ ]:
# Solutions:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

# new
markdown_text = '''
#### The bubble chart below describes the relationship between life expectancy, 
GDP and population (indicated by size of each bubble) across continents in a particular year.
'''

# The layout describes what the app looks like using HTML. The layout is a hierarchical tree of components
# For more information: https://dash.plotly.com/dash-html-components
app.layout = html.Div([

    # This is a markdown component, a Dash Core Component. DCCs are interactive components
    # For more information: https://dash.plotly.com/dash-core-components
    dcc.Markdown(children=markdown_text),                                            # new

    # This is an interactive graph component. 
    # For more information: https://dash.plotly.com/interactive-graphing & https://dash.plotly.com/dash-core-components/graph
    dcc.Graph(id="scatter-plot"),
    html.P("Select a year:"),
    dcc.Slider(1952,2007,5,
        value = 1952,
        id='year-slider',
        marks={i: '{}'.format(i) for i in range(1952,2007,5)}
    ),

    html.Br(),                                                                       # new
    html.Label("Select continents of interest in checkboxes below:"),                # new
    dcc.Checklist(options=['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'],       # new
        value=['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'],                   # new
        inline=True,                                                                 # new
        id='continent-slider'),                                                      # new
])

# This is a callback decorator, which functions to change an output based on an input 
# The component_id and component_property keywords are optional
# For more information: https://dash.plotly.com/basic-callbacks
@app.callback(
    Output(component_id="scatter-plot", component_property="figure"), 
    Input(component_id="year-slider", component_property="value"),
    Input(component_id="continent-slider", component_property="value")               # new
    )

def update_plot(year, continent):
    
    # replace with your own data source
    gm = px.data.gapminder()
    
    # using filtered data frame
    gmy = gm[(gm['year']==year) & (gm['continent'].isin(continent))]                 # new

    # standard plotly graph - plotly documentation includes template codes for Dash
    fig = px.scatter( 
        gmy, y='lifeExp', x='gdpPercap',
        size='pop', color= 'continent', hover_data = ['country'],
        log_x=True, log_y = True,
        title = 'Health vs. Wealth across the world in '+str(year)
    )
    return fig

app.run_server(debug=True)

## Task 3: Demonstration: Creating a  <code>bokeh</code> standalone dashboard

Now we are going to learn another way of using <code>bokeh</code> to produce more sophisticated dashboard.  We will start with the example from 

https://optimizemydayjob.medium.com/a-guide-to-interactive-python-dashboards-using-bokeh-2fc4904b20f2

Similar to Task 2 above, we will be using a python <code>.py</code> file to create the following dashboard:


<img src = 'tutorial7_task3.png' />

### Step 1.  Import Packages and Load the Data

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from bokeh.io import output_file, show
from bokeh.layouts import gridplot, layout
from bokeh.palettes import Category20
from bokeh.plotting import figure, curdoc
from bokeh.models import (ColumnDataSource, CDSView, RadioButtonGroup, GroupFilter, DataTable,
                          TableColumn, DateFormatter, CategoricalColorMapper, CheckboxGroup,
                          TextInput, Column, Row, Div, HoverTool, Slider, RangeSlider, MultiChoice)

# Read the data:
data = pd.read_csv(r'/Users/leesi/usyd/qbus6860/data/all_stocks_5yr.csv')

# Format the date variable to datetime and sort the data by date:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data.sort_values(by='date', ascending=False, inplace=True)

# Create the month and year variables:
data['Month'] = pd.to_datetime(data['date']).dt.month
data['Year'] = pd.to_datetime(data['date']).dt.year

As you can see, after reading in the data, we convert the <code>date</code> column into <code>Datetime</code> objects (we will learn more about this in doing time series plotting), then create two new columns <code>Month</code> and <code>Year</code>.  Please add your code to see the <code>data</code>.

### Step 2.   Get Data for Dashboard definition

Let us prepare several variables for the use in plotting

In [ ]:
# INITIAL VARIABLES:
# Create a sorted list of ticker names:
tickers = sorted(list(data.Name.unique()))
# Creat a sorted list of months:
month = sorted(list(data.Month.unique()))
# Creat a sorted list of years:
year = sorted(list(data.Year.unique()))

### Step 3.  Define Widgets

Most of time, similar to any windows, online form etc, we always see such as Button, InputField, Sliders, Droplist etc.  All these are normally called Widgets.  Many apps provide their own ways to produce to be used in an interface design.  <code>bokeh</code> defines all these tools we can use.  We have already import them at the beginning.  The following code will create a MultiChoice (or droplist), a Slider and a RangeSlider for our dashboard.

In [ ]:
# WIDGETS:
# Initialize the ticker choice:
ticker_button = MultiChoice(value=tickers[:2], options=tickers)
# Initialize the year slider:
year_slider = Slider(start=year[0], end=year[-1], value=year[1], step=1, title='Year')
# Initialize the month range slider:
month_slider = RangeSlider(start=month[0], end=month[-1], value=month[:2], step=1, title='Month')

### Step 4. Organize Data in Table to be shown on the Dashboard

Next we prepare the data for the Dashboard that we are going to create. We only pick up the data within the definition of the above three widgets.  In this table, we will only show the Data, Stock Code (Name) and Close prices.

In [ ]:
# Filter the initial data source:
df = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]
# Pass the filtered data source to the ColumnDataSource class:
source = ColumnDataSource(data=df)

# TABLE
# Creating the list of columns:
columns = [
        TableColumn(field="date", title="Date", formatter=DateFormatter(format="%Y-%m-%d")),
        TableColumn(field="Name", title="Name"),
        TableColumn(field="close", title="Close"),
    ]
# Initializing the table:
table = DataTable(source=source, columns=columns, height=500)

### Step 5.  Define Actions for Widgets 

In interactive dashboard, we shall repeatedly draw the plots. So it is better to define our own functions to be called everytime when an update is needed.  For example, <code>plot_function</code> will take the stock names from the droplist (<code>ticker</code>), and pick up the "close" prices of those stocks and draw them as lineplots.

In [ ]:
# PLOT
def plot_function(tickers):
    # Getting some colors:
    colors = list(Category20.values())[17]
    random_colors = []
    for c in range(len(tickers)):
        random_colors.append(random.choice(colors))

    # Create the hovertool:
    TOOLTIPS = HoverTool(tooltips=[    ('date', '$x{%Y-%m-%d}'),
                   ('close', '$@{close}{0.0}'),
                   ('high', '$@{high}{0.0}'),
                   ('low', '$@{low}{0.0}'),
                   ('volume', '@volume{0.00 a}')],
                         formatters={'$x': 'datetime'})

    # Create the figure to store all the plot lines in:
    p = figure(x_axis_type='datetime', width=1000, height=500)

    # Loop through the tickers and colors and create plot line for each:
    for t, rc in zip(tickers, random_colors):
        view = CDSView(source=source, filters=[GroupFilter(column_name='Name', group=t)])
        p.line(x='date', y='close', source=source, view=view, line_color=rc, line_width=4)

    # Add the hovertool to the figure:
    p.add_tools(TOOLTIPS)
    return p
p = plot_function(tickers)

def text_function(attr, old, new):
    new_text = new
    old_text = old
    text_data = pd.read_json('text_data.json')

def filter_function():
    # Filter the data according to the widgets:
    new_src = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]

    # Replace the data in the current data source with the new data:
    source.data = new_src.to_dict('series')

def change_function(attr, old, new):
    filter_function()


Now set the function (or action to be taken) to the relevant widgets.

In [ ]:
ticker_button.on_change('value', change_function)
month_slider.on_change('value', change_function)
year_slider.on_change('value', change_function)

### Step 6.  Build our Dashboard

In [ ]:
# Header
title = Div(text='<h1 style="text-align: center">Stock Dashboard</h1>')

widgets_col = Column(month_slider, year_slider)   # Two sliders are arranged in column way
widgets_row = Row(widgets_col, ticker_button) 
layout = layout([[title],
                 [widgets_row],
                 [p,table]])
curdoc().title = 'Stock Dashboard'
curdoc().add_root(layout)

The above code defines the dashboard layout:  Two sliders are arranged in column way, then on the right is the droplist (ticker_button) from which users can choose different stocks. Finally there are three rows:  A title row at the top showing a title, then the first widgets (two sliders and a droplist), and the last row with a plot (<code>bokeh</code> figure object) and a table.

### Step 7.  Take it in action

If you run the above code step by step, you will see nothing happens in the end.  The reason is, the information can only be shown through the <code>bokeh</code> server. You may follow the following steps to make it happen.

How to test this dashboard.

1. Download the data file <code>all_stocks_5yr.csv</code>
2. Copy the following code in the following cell (I put all the above code together) into a py file, for example, name it as <code>mydashboard.py</code>
3. From Anaconda Navigator, go to your Environment's terminal window (please refer to SoftwareInstruction document). In my case it is 
<code> cd C:\Users\leesi\usyd\qbus6860\2022S2 </code>
4. In the terminal window, cd to the folder where you save the <code>mydashboard.py</code> and <code>all_stocks_5yr.csv</code>, then type in the following command
      <code>bokeh serve --show mydashboard.py</code>

5. A new tab window will be popped out in your browser where you will have the dashboard. See the image at the bottom

In [ ]:
# The codes below are not meant for execution in Jupyter
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from bokeh.io import output_file, show
from bokeh.layouts import gridplot, layout
from bokeh.palettes import Category20
from bokeh.plotting import figure, curdoc
from bokeh.models import (ColumnDataSource, CDSView, RadioButtonGroup, GroupFilter, DataTable,
                          TableColumn, DateFormatter, CategoricalColorMapper, CheckboxGroup,
                          TextInput, Column, Row, Div, HoverTool, Slider, RangeSlider, MultiChoice)
# Read the data:
data = pd.read_csv('all_stocks_5yr.csv')
# Format the date variable to datetime and sort the data by date:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data.sort_values(by='date', ascending=False, inplace=True)
# Create the month and year variables:
data['Month'] = pd.to_datetime(data['date']).dt.month
data['Year'] = pd.to_datetime(data['date']).dt.year

# INITIAL VARIABLES:
# Create a sorted list of ticker names:
tickers = sorted(list(data.Name.unique()))
# Creat a sorted list of months:
month = sorted(list(data.Month.unique()))
# Creat a sorted list of years:
year = sorted(list(data.Year.unique()))

# WIDGETS:
# Initialize the ticker choice:
ticker_button = MultiChoice(value=tickers[:2], options=tickers)
# Initialize the year slider:
year_slider = Slider(start=year[0], end=year[-1], value=year[1], step=1, title='Year')
# Initialize the month range slider:
month_slider = RangeSlider(start=month[0], end=month[-1], value=month[:2], step=1, title='Month')


# Filter the initial data source:
df = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]
# Pass the filtered data source to the ColumnDataSource class:
source = ColumnDataSource(data=df)

# TABLE
# Creating the list of columns:
columns = [
        TableColumn(field="date", title="Date", formatter=DateFormatter(format="%Y-%m-%d")),
        TableColumn(field="Name", title="Name"),
        TableColumn(field="close", title="Close"),
    ]
# Initializing the table:
table = DataTable(source=source, columns=columns, height=500)

# PLOT
def plot_function(tickers):
    # Getting some colors:
    colors = list(Category20.values())[17]
    random_colors = []
    for c in range(len(tickers)):
        random_colors.append(random.choice(colors))

    # Create the hovertool:
    TOOLTIPS = HoverTool(tooltips=[    ('date', '$x{%Y-%m-%d}'),
                   ('close', '$@{close}{0.0}'),
                   ('high', '$@{high}{0.0}'),
                   ('low', '$@{low}{0.0}'),
                   ('volume', '@volume{0.00 a}')],
                         formatters={'$x': 'datetime'})

    # Create the figure to store all the plot lines in:
    p = figure(x_axis_type='datetime', width=1000, height=500)

    # Loop through the tickers and colors and create plot line for each:
    for t, rc in zip(tickers, random_colors):
        view = CDSView(source=source, filters=[GroupFilter(column_name='Name', group=t)])
        p.line(x='date', y='close', source=source, view=view, line_color=rc, line_width=4)

    # Add the hovertool to the figure:
    p.add_tools(TOOLTIPS)
    return p
p = plot_function(tickers)

def text_function(attr, old, new):
    new_text = new
    old_text = old
    text_data = pd.read_json('text_data.json')

def filter_function():
    # Filter the data according to the widgets:
    new_src = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]

    # Replace the data in the current data source with the new data:
    source.data = new_src.to_dict('series')

def change_function(attr, old, new):
    filter_function()

ticker_button.on_change('value' ,change_function)
month_slider.on_change('value', change_function)
year_slider.on_change('value', change_function)

# Header
title = Div(text='<h1 style="text-align: center">Stock Dashboard</h1>')

widgets_col = Column(month_slider, year_slider)
widgets_row = Row(widgets_col, ticker_button)
layout = layout([[title],
                 [widgets_row],
                 [p,table]])
curdoc().title = 'Stock Dashboard'
curdoc().add_root(layout)

<hr style="border:1px solid gray"> <center><h1> Alternative Tasks </h1></center><hr style="border:1px solid gray">

## Task: Simple Dashboard in Notebook using <code>pandas_bokeh</code>

In this task, we will demonstrate how to build an interactive dashboard. The code is adopted from https://www.analyticsvidhya.com/blog/2021/09/building-an-interactive-dashboard-using-bokeh-and-pandas/

To run this notebook, you must install <code>pandas_bokeh</code>. If you have not installed it, you can do the following pip install <code>pandas_bokeh</code>.

### Step 1.  Import the packages

Remember to import <code>pandas</code>, <code>numpy</code> etc before the <code>pandas_bokeh</code> library.

In [ ]:
# conda install -c patrikhlobil pandas-bokeh

In [ ]:
import numpy as np
import pandas as pd
import pandas_bokeh 

Prepare for plots to be shown inside Notebook.  However if you wish to produce html file for the dashboard, use the following

<code>pandas_bokeh.output_file('mydashboard.html')</code>

In [ ]:
pandas_bokeh.output_notebook()

### Step 2. Produce some simulated data to draw

In [ ]:
#define the categorical variable
category = ['A','B','C']
#set random seed to make the dataset reproducible
np.random.seed(42)
#create a dataset
df_random = pd.DataFrame({
   'id': np.arange(0, 15),
   'month':np.random.randint(1, 12, 15),
   'sensor_1': np.random.uniform(0, 1,15),
   'sensor_2': np.random.uniform(10, 15, 15),
   'sensor_3': np.random.randint(0, 20, 15),
   'category': np.random.choice(category, 15, p=[0.2, 0.4, 0.4])
})

#set index to id column
df_random=df_random.set_index('id')
df_random

### Step 3.   Draw Plots

Let us plot the following charts using the pandas_bokeh library-

    Line plot
    Bar chart
    Stacked Bar chart
    Scatter plot
    Pie chart
    Histogram

Also please think about how these plotting function of <code>bokeh</code> under <code>pandas</code> are different from plotting function in <code>seaborn</code>.

In [ ]:
# Plot1 - Line plot
p_line= df_random.groupby(['month']).mean().plot_bokeh(kind="line",y="sensor_2",color='#d01c8b',plot_data_points=True,show_figure=False)

# Plot2 - Barplot
p_bar = df_random.groupby(['month']).mean().plot_bokeh(kind="bar",show_figure=False)   

# Plot3 - stacked bar chart
df_sensor=df_random.drop(['month'],axis=1)
p_stack=df_sensor.groupby(['category']).mean().plot_bokeh(kind='barh', stacked=True,show_figure=False)  

#Plot4 - Scatterplot
p_scatter = df_random.plot_bokeh(kind="scatter", x="month", y="sensor_2",category="category",show_figure=False)  

#Plot5 - Pie chart
p_pie= df_random.groupby(['category']).mean().plot_bokeh.pie(y='sensor_1',show_figure=False)  

#Plot6 - Histogram
p_hist=df_sensor.plot_bokeh(kind='hist', histogram_type="stacked",bins=6,show_figure=False)  

### Step 4. Make the Dashboard

In [ ]:
#Make Dashboard with Grid Layout: 
pandas_bokeh.plot_grid([[p_line, p_bar,p_stack],[p_scatter, p_pie,p_hist]], plot_width=300)